In [ ]:
import numpy as np
import pdb

from sklearn.datasets import make_classification

import cv2



In [ ]:
index = 0
def extract_image_features(image):
    global index
    sift = cv2.xfeatures2d.SIFT_create()
    image_bgr = cv2.cvtColor(image.T, cv2.COLOR_RGB2BGR)
    _ , descriptors =  sift.detectAndCompute(image_bgr, None)
    try:
        descriptors.shape
    except AttributeError:
        print(index)
        index = index + 1
        return np.empty([0, 128])
    index = index + 1
    return descriptors

def process_images(images):
    return np.concatenate([extract_image_features(image) for image in images])

In [ ]:
def fisher_vector(xx, gmm):
    """Computes the Fisher vector on a set of descriptors.
    Parameters
    ----------
    xx: array_like, shape (N, D) or (D, )
        The set of descriptors
    gmm: instance of sklearn mixture.GMM object
        Gauassian mixture model of the descriptors.
    Returns
    -------
    fv: array_like, shape (K + 2 * D * K, )
        Fisher vector (derivatives with respect to the mixing weights, means
        and variances) of the given descriptors.
    Reference
    ---------
    J. Krapac, J. Verbeek, F. Jurie.  Modeling Spatial Layout with Fisher
    Vectors for Image Categorization.  In ICCV, 2011.
    http://hal.inria.fr/docs/00/61/94/03/PDF/final.r1.pdf
    """
    xx = np.atleast_2d(xx)
    N = xx.shape[0]

    # Compute posterior probabilities.
    Q = gmm.compute_posteriors(xx)  # NxK
    
    Q = Q.asarray()

    # Compute the sufficient statistics of descriptors.
    Q_sum = np.sum(Q, 0)[:, np.newaxis] / N
    Q_xx = np.dot(Q.T, xx) / N
    Q_xx_2 = np.dot(Q.T, xx ** 2) / N

    # Compute derivatives with respect to mixing weights, means and variances.
    d_pi = Q_sum.squeeze() - gmm.get_weights()
    d_mu = Q_xx - Q_sum * gmm.get_means()
    d_sigma = (
        - Q_xx_2
        - Q_sum * gmm.get_means() ** 2
        + Q_sum * gmm.get_covars()
        + 2 * Q_xx * gmm.get_means())

    # Merge derivatives into a vector.
    return np.hstack((d_pi, d_mu.flatten(), d_sigma.flatten()))

In [ ]:
from scipy import ndimage, misc
import numpy as np
import os
import pandas as pd
from pandas import HDFStore, DataFrame

import h5py

In [ ]:
dataset_dir = ('../dataset_h5/')

h5f = h5py.File(os.path.join(dataset_dir,'images_224_delta_1.5.h5'),'r')

In [ ]:
##EXAMPLE OF IMAGES WITH NO SIFT POINTS
%matplotlib inline
from matplotlib import pyplot as plt
plt.imshow(h5f['data'][144].T, interpolation='nearest')
plt.figure()
plt.imshow(h5f['data'][406].T, interpolation='nearest')
plt.figure()
plt.imshow(h5f['data'][609].T, interpolation='nearest')
plt.figure()
plt.imshow(h5f['data'][732].T, interpolation='nearest')
plt.figure()
plt.imshow(h5f['data'][869].T, interpolation='nearest')
plt.figure()
plt.imshow(h5f['data'][923].T, interpolation='nearest')
plt.show()

In [ ]:
#image_features = process_images(h5f['data'])
# sift.detectAndCompute(image_bgr, None)
# cv2.xfeatures2d.SIFT_create().detectAndCompute(h5f['data'][0], None)
#_, image_features = sift.detectAndCompute(image_bgr, None)

#h5f['data'][144]
# cv2.imshow('image',h5f['data'][144])
filename = 'descriptors_images_224_delta_1.5.h5'
if(os.path.isfile(os.path.join(dataset_dir,filename))):
    descriptor_h5f = h5py.File(os.path.join(dataset_dir,filename),'r')
    image_descriptors = descriptor_h5f['descriptors']
else:
    descriptor_h5f = h5py.File(os.path.join(dataset_dir,filename),'w')
    image_descriptors = process_images(h5f['data'][:10000])
    descriptor_h5f['descriptors'] = image_descriptors
# image_descriptors.shape


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=64)# adjust yourself
pca.fit(image_descriptors)
image_descriptors_reduced = pca.transform(image_descriptors)
# X_t_test = pca.transform(X_test)


In [ ]:
image_descriptors_reduced.shape

In [ ]:
descriptor_h5f['descriptor_reduced'] = image_descriptors_reduced

In [ ]:
from scipy import ndimage, misc
import numpy as np
import os
import pandas as pd
from pandas import HDFStore, DataFrame

import h5py

dataset_dir = ('../dataset_h5/')
filename = 'descriptors_images_224_delta_1.5.h5'
descriptor_h5f = h5py.File(os.path.join(dataset_dir,filename),'r')
image_descriptors_reduced = descriptor_h5f['descriptor_reduced'][:]

In [ ]:
# full_dataset = h5f['data'][:]
# full_dataset = full_dataset.reshape(full_dataset.shape[0], -1)
# randomly_sampled = np.random.choice(full_dataset.shape[0], size=10000, replace=False)

# X_train = X_train.reshape(X_train.shape[0], -1)
# X_test = h5f['data'][-10000: ]
# X_test = X_test.reshape(X_test.shape[0], -1)

import ggmm.gpu as ggmm

N, D = image_descriptors_reduced.shape
K=128

ggmm.init(2731155)
gmm = ggmm.GMM(K,D)


thresh = 1e-3 # convergence threshold
n_iter = 20 # maximum number of EM iterations
init_params = 'wmc' # initialize weights, means, and covariances

# train GMM
gmm.fit(image_descriptors_reduced, thresh, n_iter, init_params=init_params)

# retrieve parameters from trained GMM
weights = gmm.get_weights()
means = gmm.get_means()
covars = gmm.get_covars()

# compute posteriors of data
posteriors = gmm.compute_posteriors(image_descriptors_reduced)


In [ ]:
fv = fisher_vector(image_descriptors_reduced, gmm)

In [3]:
from scipy import ndimage, misc
import numpy as np
import os
import pandas as pd
from pandas import HDFStore, DataFrame

import h5py

dataset_dir = ('../dataset_h5')
filename = 'descriptors_images_224_delta_1.5.h5'
descriptor_h5f = h5py.File(os.path.join(dataset_dir,filename),'r')


fv = descriptor_h5f['fisher_vector']

fv.shape

(16512,)

In [5]:
fv[:]

array([ -2.79233216e-05,  -8.29181364e-06,   5.64052493e-05, ...,
         1.52099688e-02,   2.96574679e-02,   1.00977729e-02])

In [ ]:
def train(gmm, features):
    X = features
    Y = 

    clf = svm.SVC()
    clf.fit(X, Y)
    return clf

In [ ]:
classifier = train(gmm, fv)
# rate = success_rate(classifier, fv)
# print("Success rate is", rate)

In [ ]:
descriptors = cv2.SIFT().detectAndCompute(full_dataset[0], None)